In [ ]:
wd<-"/result/Seurat/sc_Astrocyte_recluster/lee/"
if(!dir.exists(wd))
    dir.create(wd)
setwd(wd)
getwd()

In [ ]:
#library(spacexr)
library(Matrix)
#library(doParallel)
library(Seurat)
library(ggplot2)
library(patchwork)
library(dplyr)
#library(hdf5r)
library(png)
#library(ggalluvial)
#library(tidyverse)
library(clustree)
library(RCTD)
library(Matrix)
#library(SPOTlight)
library(future)
library(grid)
library(RColorBrewer)
library(pheatmap)
#library(circlize)
library(igraph)
#library(ComplexHeatmap)
library(stringr)
#library(ComplexHeatmap)
library(cowplot)
library(scales)
#library(dplyr)
#library(tidyverse)
library(reshape2)
library(clusterProfiler)
options(connectionObserver = NULL)
library("readxl")

In [ ]:
spatial_marker<-read.csv("/result/CSIDE/marker_express_within_region.220426/fcreg1_220503/2subtype.marker.sig.enrich.in.sham.2domains.df.csv")
head(spatial_marker)
table(spatial_marker$group)

In [ ]:
da<-readRDS("/result/RCTD/ref_10x_astr/L3_taxon/Astrocyte_integrate_split2A/splitpool_3astr.labeltransfer.10x_L3.astrocyte.labelAstrcoyte.object.with7dpi.rds")
da

In [3]:
da2<-readRDS("/data/deconvolution/sci.rds")

#### Astrocyte migration marker score -220613

In [1319]:
da2$SCT_Stat3<-da2@assays$SCT@data["Stat3",]
da2$SCT_Socs3<-da2@assays$SCT@data["Socs3",]
da2$SCT_Itgb4<-da2@assays$SCT@data["Itgb4",]
da2$SCT_Nes<-da2@assays$SCT@data["Nes",]
da2$SCT_C3<-da2@assays$SCT@data["C3",]

In [ ]:
options(repr.plot.width=6,repr.plot.height=5)
DimPlot(da2,cols = DiscretePalette(n = 15,palette = "alphabet"))
ggsave("L2.celltype.all.integrated.UMAP.png",width = 6,height = 5,dpi = 300)

In [ ]:
options(repr.plot.width=5,repr.plot.height=5)
FeaturePlot(da2,pt.size = 0.01,ncol = 1,#cells = cells,
            features = c("Nrxn1"#,"Gfap","Slc7a10"
                        ),
           cols= c(#"#cfe0e8",
            
                                                     # '#b8a9c9', 
            '#d6d4e0',
            "#f0f0f0",
            '#fff2df',
            "#eeac99",#"#ED797B",
                     "#c83349")
           )
ggsave("Nrxn1.all.sc.integrated.data.UMAP.png",width = 5,height = 5,dpi = 300)

In [ ]:
options(repr.plot.width=9,repr.plot.height=5)
FeaturePlot(da2,pt.size = 0.01,ncol = 2,#cells = cells,
            features = c("Nlgn1","Nlgn2","Nlgn3"
                        ),
           cols= c(#"#cfe0e8",
            
                                                     # '#b8a9c9', 
            '#d6d4e0',
            "#f0f0f0",
            '#fff2df',
            "#eeac99",#"#ED797B",
                     "#c83349")
           )
ggsave("Nlgn1_3.all.sc.integrated.data.UMAP.png",width = 9,height = 5,dpi = 300)

#### add Astrocyte subtype info

In [ ]:
gfap_cells<-rownames(test@meta.data)[test$subtype=="Astr_Gfap"]
length(gfap_cells)
slc_cells<-rownames(test@meta.data)[test$subtype=="Astr_Slc7a10"]
length(slc_cells)

In [ ]:
da2$celltype_ad<-as.character(da2$celltype)
da2$celltype_ad[rownames(da2@meta.data) %in% gfap_cells]<-"Astr_Gfap"
da2$celltype_ad[rownames(da2@meta.data) %in% slc_cells]<-"Astr_Slc7a10"
table(da2$celltype_ad)

In [17]:
saveRDS(da2,"lee.sci.ad.rds")

In [6]:
da2<-readRDS("lee.sci.ad.rds")

In [8]:
DefaultAssay(da2)<-"SCT"

In [475]:
sfdir<-"Astrocyte_gfap.marker.across.sc/"
if(!dir.exists(sfdir))
    dir.create(sfdir)

In [470]:
gfap_markers<-c("Gfap","Clu","Aqp4","Ndrg2","Cpe","Fxyd1",
                   "Gja1","Dhrs1","Dclk1","Lxn","Ldhb","Mmd2",
                  "Gpm6a","Pea15a","Nrcam","A2m","Scg3","Itpkb",
                  "Tmem47","Itgav","Mgst1","C3","Id4","Agt","Prdx6",
               "Il6st","Nbl1","Mt2","Cd63","Pea15a","Lgals1","Ifitm3",
               "Vwa1","Tmem176b","Ctsl")

In [479]:
options(repr.plot.width=5,repr.plot.height=5)
for(i in gfap_markers){
    if(i %in% rownames(da2)){
        FeaturePlot(da2,pt.size = 0.01,ncol = 1,#cells = cells,
                features = c(i#,"Gfap","Slc7a10"
                            ),
               cols= c(#"#cfe0e8",
            
                                                     # '#b8a9c9', 
                '#d6d4e0',
                "#f0f0f0",
                '#fff2df',
                "#eeac99",#"#ED797B",
                "#c83349")
               )
    ggsave(paste0(sfdir,i,".all.sc.integrated.data.UMAP.png"),width = 5,height = 5,dpi = 300)
    }    
}

In [480]:
options(repr.plot.width=6,repr.plot.height=5)
for(i in gfap_markers){
    if(i %in% rownames(da2)){
        VlnPlot(da2,group.by = "celltype_ad",ncol = 1,
                features=c(i),
                pt.size = 0)+
            geom_boxplot(width=0.25,fill="white",outlier.colour = NA)
        ggsave(paste0(sfdir,i,".all.sc.integrated.data.vlnplot.png"),width = 5,height = 5,dpi = 300)
    }
    
}


In [4]:
DefaultAssay(da2)<-"SCT"

In [1326]:
da2<-AddModuleScore(da2,features = list(c("Socs3","Itgb4","Stat3","Nes","C3","Itgb4")),name = "migration_marker")

In [ ]:
DefaultAssay(da2)<-"integrated"
colnames(da2@meta.data)

In [ ]:
options(repr.plot.width=5,repr.plot.height=5)
FeaturePlot(da2,pt.size = 0.1,#cells = cells,
            features = c("Igfbp2"
                        )#,
           #cols= c(#"#cfe0e8",
           # 
           #                                          # '#b8a9c9', 
           # '#d6d4e0',
           # "#f0f0f0",
           # '#fff2df',
           # "#eeac99",#"#ED797B",
           #          "#c83349")
           )

In [ ]:
table(da2@meta.data$celltype)
DefaultAssay(da2)<-"RNA"

In [ ]:
test<-subset(da2,subset = celltype=="Astrocyte")
test

In [1057]:
da_s<-CreateSeuratObject(counts = test@assays$RNA@counts,meta.data = test@meta.data,min.cells = 10)

In [ ]:
rps_genes<-rownames(da_s@assays$RNA@counts)[grep("^Rps",rownames(da_s@assays$RNA@counts))]
rps_genes
rpl_genes<-rownames(da_s@assays$RNA@counts)[grep("^Rpl",rownames(da_s@assays$RNA@counts))]
rpl_genes
mt_genes<-rownames(da_s@assays$RNA@counts)[grep("^mt-",rownames(da_s@assays$RNA@counts))]
mt_genes

In [ ]:
# remove ribosome and mitocondrial genes
da_s<-da_s[!rownames(da_s@assays$RNA@counts)%in%c(rps_genes,rpl_genes,mt_genes),rownames(da_s@meta.data)]
da_s

In [ ]:
### QC
# Visualize QC metrics as a violin plot
options(repr.plot.width=10,repr.plot.height=5)
VlnPlot(da_s, features = c("nFeature_RNA", "nCount_RNA", "percent_mt","percent_rp","percent_hbb"), ncol = 5)

In [ ]:
plot1 <- FeatureScatter(da_s, feature1 = "nCount_RNA", feature2 = "percent_mt")
plot2 <- FeatureScatter(da_s, feature1 = "nCount_RNA", feature2 = "nFeature_RNA")
plot1 + plot2

In [ ]:
da_s<-SCTransform(da_s,do.scale = TRUE,
                  vars.to.regress = c("orig.ident","S.Score","G2M.Score","percent_mt","percent_rp"
                                     ),
                  residual.features = rownames(da_s@assays$RNA@counts),
                    return.only.var.genes = FALSE)

In [1276]:
saveRDS(da_s,"L2.Astrocyte.rmMt_Rp.SCT.regICMR.doscale.rds")

In [ ]:
cells<-rownames(da_s@meta.data)[da_s$nCount_RNA>2500]
length(cells)

In [ ]:
test<-da_s[rownames(da_s@assays$RNA@counts),cells]
test

In [1213]:
test<-FindVariableFeatures(test)

In [ ]:
test<-RunPCA(test#,features = astr_marker
              )

In [ ]:
test<-FindNeighbors(test,dims=1:16,k.param = 40)
for(i in c(0.1,0.125,0.15,0.2,0.3,0.35,0.4,0.45,0.5)){
    test<-FindClusters(test,resolution = i)
}


In [ ]:
options(repr.plot.width=6,repr.plot.height=8)
clustree(test,prefix = "SCT_snn_res.")

In [ ]:
test<-RunUMAP(test,dims = 1:16)

In [ ]:
options(repr.plot.width=10,repr.plot.height=4)
DimPlot(test,reduction = "umap",#cols = DiscretePalette(n=10,palette = "glasbey"),#cells = cells,
        group.by = c("SCT_snn_res.0.35","SCT_snn_res.0.2","time"))

In [ ]:
options(repr.plot.width=3.6,repr.plot.height=4)
DimPlot(test,reduction = "umap",#cols = DiscretePalette(n=10,palette = "glasbey"),#cells = cells,
        group.by = c("time"))+
     scale_color_manual(values = c('#374E55FF','#00A1D5FF','#B24745FF','#d6cbd3'
))
ggsave("L2.Astrocyte.nC2500.rmMt_Rp.SCTregICMR.doscale.pc16.k40.time.UMAP.png",width = 3.6,height = 4,dpi = 300)

In [ ]:
options(repr.plot.width=3,repr.plot.height=4)
DimPlot(test,reduction = "umap",#cols = DiscretePalette(n=10,palette = "glasbey"),#cells = cells,
        group.by = c("SCT_snn_res.0.2"))+
     scale_color_manual(values = c('#911eb4','#e6beff'))
ggsave("L2.Astrocyte.nC2500.rmMt_Rp.SCTregICMR.doscale.pc16.k40.res02.2subtype.UMAP.png",width = 3,height = 4,dpi = 300)

In [ ]:
options(repr.plot.width=10,repr.plot.height=4)
VlnPlot(test,group.by = "SCT_snn_res.0.35",
        features=c("nCount_RNA","nFeature_RNA","percent_mt"),
        pt.size = 0)

In [ ]:
pdf("lee.Astrocyte.Pla2g7_Slc7a10_Gfap_Igfbp2.UMAP.220820.pdf",width = 14,height = 5)
options(repr.plot.width=14,repr.plot.height=5)
FeaturePlot(test,pt.size = 1,ncol = 4,#cells = cells,
            features = c("Gfap","Pla2g7",
                         #"Slc1a2","Aldoc","Bcan","Clu","S100b","Aqp4",
                         "Slc7a10","Igfbp2"#,"Mdk","Itpkb"
                        ),
           cols= c(#"#cfe0e8",
    
                                                     # '#b8a9c9', 
    '#d6d4e0',
    "#f0f0f0",
    '#fff2df',
    "#eeac99",#"#ED797B",
                     "#c83349")
           )
dev.off()
#ggsave("L2.Astrocyte.nC2500.rmMt_Rp.SCTregICMR.doscale.pc16.k40.Gfap_Slc7a10_Vim.data.UMAP.png",width = 12,height = 5,dpi = 300)

In [ ]:
options(repr.plot.width=12,repr.plot.height=5)
FeaturePlot(test,pt.size = 1,ncol = 3,#cells = cells,
            features = c("Gfap","Slc7a10","Bcan"
                        ),
           cols= c(#"#cfe0e8",
    
                                                     # '#b8a9c9', 
    '#d6d4e0',
    "#f0f0f0",
    '#fff2df',
    "#eeac99",#"#ED797B",
                     "#c83349")
           )
#ggsave("L2.Astrocyte.nC2500.rmMt_Rp.SCTregICMR.doscale.pc16.k40.Gfap_Slc7a10_Vim.data.UMAP.png",width = 12,height = 5,dpi = 300)

In [ ]:
pdf("lee.Astrocyte.Pla2g7_Slc7a10_Gfap_Igfbp2.Vlnplot.220820.pdf",width = 10,height = 4)
options(repr.plot.width=10,repr.plot.height=4)
VlnPlot(test,group.by = "subtype",ncol = 5,cols = c('#911eb4','#e6beff'),
        features=c("Gfap","Slc7a10","Pla2g7","Igfbp2"),
        pt.size = 0)#+
        #scale_fill_manual(values = c('#911eb4','#e6beff'))
dev.off()
#ggsave("L2.Astrocyte.nC2500.rmMt_Rp.SCTregICMR.doscale.pc16.k40.res02.some_genes.UMAP.png",width = 10,height = 6,dpi = 300)

In [ ]:
# take a look at the spatial related genelist
dp_genes<-read.csv("/home/jovyan/zxli_SCI/result/Seurat/sc_Astrocyte_recluster/lee/classifyBySpatialMarker/gfap.317marker.k4.m13.hclust2.fc1.pct05.pad001.pos.markers.csv")
dp_genes<-dp_genes$gene
head(dp_genes)

In [7]:
test<-AddModuleScore(test,features = list(dp_genes),name = "dp_marker")

In [156]:
test<-AddModuleScore(test,features = list(dp_genes2),name = "gfap_dp_marker")

#### migration score

In [1292]:
test<-AddModuleScore(test,features = list(c("Stat3","Socs3","Itgb4","Nes","C3")),name = "migration_marker")

In [ ]:
#positive regulon of cell migration term genes
cm<-read_excel("classifyBySpatialMarker/GO_term_positiveregulationofcellmigration_20220620_052409.xlsx")
head(cm)
cmgenes<-unique(cm$Symbol)
length(cmgenes)

In [ ]:
cmgenes<-intersect(cmgenes,rownames(test@assays$SCT@data))
length(cmgenes)

In [492]:
test<-AddModuleScore(test,features = list(cmgenes),name = "positive_cell_migration")

In [ ]:
options(repr.plot.width=4,repr.plot.height=4)
VlnPlot(test,group.by = "SCT_snn_res.0.2",cols = c('#911eb4','#e6beff'),
        features=c("dp_marker1"),
        pt.size = 0)+
    geom_boxplot(width=0.25,fill="white",outlier.colour = NA)
ggsave("L2.Astrocyte.nC2500.rmMt_Rp.SCTregICMR.doscale.pc16.k40.res02.2subtype.GM_Gfap_both_pos_deg_score.UMAP.png",width = 3.6,height = 4,dpi = 300)

In [ ]:
test$subtype<-"Astr_Gfap"
test$subtype[test$SCT_snn_res.0.2==1]<-"Astr_Slc7a10"
table(test$subtype)

In [ ]:
#Idents(test)<-test$subtype
options(repr.plot.width=4,repr.plot.height=4)
VlnPlot(test,group.by = "subtype",
        features=c("positive_cell_migration1"),
        pt.size = 0)+
    geom_boxplot(width=0.35,fill="white",outlier.colour = NA)+
    scale_fill_manual(values = c('#911eb4','#e6beff'))
ggsave("L2.Astrocyte.nC2500.rmMt_Rp.SCTregICMR.doscale.pc16.res035.k40_gfaphclust.all.positive_cell_migration_score.across.subtype.vlnplot.png",width = 4,height = 4,dpi = 300)

In [ ]:
options(repr.plot.width=3.6,repr.plot.height=5)
FeaturePlot(test,pt.size = 1,#cells = cells,
            features = c("migration_marker1"#,"Mdk","Itpkb"
                        ),
           cols= c(#"#cfe0e8",
    
                                                     # '#b8a9c9', 
    '#d6d4e0',
    #"#f0f0f0",
    '#fff2df',
    "#eeac99",#"#ED797B",
                     "#c83349")
           )

In [ ]:
pdf("astro.2subtypes.2marker.vlnplot_220913.pdf",width = 8,height = 4)
options(repr.plot.width=8,repr.plot.height=4)
VlnPlot(test,group.by = "subtype",ncol = 4,
        features=c("Pla2g7","Slc7a10","Gfap","Dhrs1"),
        pt.size = 0)
dev.off()

In [ ]:
gfap_me<-read.csv("/home/jovyan/zxli_SCI/result/Seurat/sc_Astrocyte_recluster/lee/classifyBySpatialMarker/gfap.317_bi_m13.hclust2.cluster.metadata.csv")
rownames(gfap_me)<-gfap_me[,1]
gfap_me<-gfap_me[,-1]
head(gfap_me)

In [ ]:
test$gfap_hclust<-NA
test@meta.data[rownames(gfap_me),"gfap_hclust"]<-gfap_me$hcluster
table(test$gfap_hclust)

In [510]:
saveRDS(test,"L2.Astrocyte.nC2500.rmMt_Rp.SCTregICMR.doscale.pc16.res035.k40_gfaphclust.all.rds")

In [3]:
test<-readRDS("L2.Astrocyte.nC2500.rmMt_Rp.SCTregICMR.doscale.pc16.res035.k40_gfaphclust.all.rds")

#### subset gfap cells

In [ ]:
test_g<-subset(test,subset=SCT_snn_res.0.2==0)
test_g

In [45]:
# remove small cluster(pc16, res01)
test_g<-subset(test_g,subset= SCT_snn_res.0.1 ==0)

In [46]:
test_g<-FindVariableFeatures(test_g)

In [ ]:
test_g<-RunPCA(test_g#,features = astr_marker
              )

In [ ]:
test_g<-FindNeighbors(test_g,dims=1:10,k.param = 20
                     )
for(i in c(0.1,0.125,0.15,0.2,0.3,0.35,0.4,0.45,0.5)){
    test_g<-FindClusters(test_g,resolution = i)
}


In [ ]:
options(repr.plot.width=6,repr.plot.height=8)
clustree(test_g,prefix = "SCT_snn_res.")

In [ ]:
test_g<-RunUMAP(test_g,dims = 1:10)

In [ ]:
options(repr.plot.width=10,repr.plot.height=4)
DimPlot(test_g,reduction = "umap",#cols = DiscretePalette(n=10,palette = "glasbey"),#cells = cells,
        group.by = c("SCT_snn_res.0.4","SCT_snn_res.0.3","time"))#+
    #scale_color_manual(values = c('#f2e394','#d5f4e6','#80ced6','#eea29a'))

In [ ]:
options(repr.plot.width=3.5,repr.plot.height=4)
DimPlot(test_g,reduction = "umap",#cols = DiscretePalette(n=10,palette = "glasbey"),#cells = cells,
        group.by = c("SCT_snn_res.0.4"))+
    scale_color_manual(values = c('#618685',
                                  '#f2e394',
        #'#d5f4e6',
        
        '#f9d5e5',
        '#80ced6',
        '#eea29a'
        
        
        
        #,
                                  
                                  #,'#d6cbd3','#a2836e'
                                 ))
ggsave("Astrocyte.Gfap.subset.pc10.res04.5clusters.UMAP.png",width = 3.5,height = 4,dpi = 300)

In [ ]:
options(repr.plot.width=4,repr.plot.height=4)
DimPlot(test_g,reduction = "umap",#cols = DiscretePalette(n=10,palette = "glasbey"),#cells = cells,
        group.by = c("time"))+
    scale_color_manual(values = c('#374E55FF','#00A1D5FF',
                                  '#B24745FF',
                                  '#d6cbd3'

                                 ))
ggsave("Astrocyte.Gfap.subset.pc10.res04.time.UMAP.png",width = 3.5,height = 4,dpi = 300)

In [ ]:
options(repr.plot.width=10,repr.plot.height=4)
DimPlot(test_g,reduction = "umap",#cols = DiscretePalette(n=10,palette = "glasbey"),#cells = cells,
        group.by = c("SCT_snn_res.0.1","SCT_snn_res.0.3","time"))#+
    #scale_color_manual(values = c('#f2e394','#d5f4e6','#80ced6','#eea29a'))

In [158]:
test_g<-AddModuleScore(test_g,features = list(dp_genes),name = "dp_marker")
test_g<-AddModuleScore(test_g,features = list(dp_genes2),name = "dp2_marker")

In [ ]:
options(repr.plot.width=4,repr.plot.height=5)
FeaturePlot(test_g,pt.size = 1,ncol = 1,#cells = cells,
            features = c("dp_marker1"#,"Mdk","Itpkb"
                        ),
           cols= c(#"#cfe0e8",
    
                                                     # '#b8a9c9', 
    '#d6d4e0',
    "#f0f0f0",
    '#fff2df',
    "#eeac99",#"#ED797B",
                     "#c83349")
           )
ggsave("Astrocyte.Gfap.subset.pc10.res04.GM_Gfap_both_deg_score.UMAP.png",width = 4,height = 5,dpi = 300)

In [ ]:
options(repr.plot.width=3.5,repr.plot.height=4)
VlnPlot(test_g,group.by = "SCT_snn_res.0.4",
        features = c('dp_marker1'),ncol = 1,
        col = c('#618685',
         '#f2e394',
        '#f9d5e5',
        '#80ced6',
        '#eea29a'),
        #slot = "scale.data",
        pt.size = 0)+
    geom_boxplot(width=0.2,fill="white",outlier.colour = NA)
ggsave("Astrocyte.Gfap.subset.pc10.res04.GM_Gfap_both_deg_score.5clusters.vlnplot.png",width = 3.5,height = 4,dpi = 300)

In [ ]:
options(repr.plot.width=3.5,repr.plot.height=4)
VlnPlot(test_g,group.by = "SCT_snn_res.0.4",
        features = c('Nrxn1'),ncol = 1,
        col = c('#618685',
         '#f2e394',
        '#f9d5e5',
        '#80ced6',
        '#eea29a'),
        #slot = "scale.data",
        pt.size = 0)+
    geom_boxplot(width=0.2,fill="white",outlier.colour = NA)
ggsave("Astrocyte.Gfap.subset.pc10.res04.Nrxn1.data.vlnplot.png",width = 3.5,height = 4,dpi = 300)

In [ ]:
options(repr.plot.width=3.5,repr.plot.height=4)
VlnPlot(test_g,group.by = "SCT_snn_res.0.4",
        features = c('Nrxn2'),ncol = 1,
        col = c('#618685',
         '#f2e394',
        '#f9d5e5',
        '#80ced6',
        '#eea29a'),
        #slot = "scale.data",
        pt.size = 0)+
    geom_boxplot(width=0.2,fill="white",outlier.colour = NA)

In [ ]:
options(repr.plot.width=3.5,repr.plot.height=4)
VlnPlot(test_g,group.by = "SCT_snn_res.0.4",
        features = c('Slc7a10'),ncol = 1,
        col = c('#618685',
         '#f2e394',
        '#f9d5e5',
        '#80ced6',
        '#eea29a'),
        #slot = "scale.data",
        pt.size = 0)+
    geom_boxplot(width=0.2,fill="white",outlier.colour = NA)
ggsave("Astrocyte.Gfap.subset.pc10.res04.Slc7a10.data.vlnplot.png",width = 3.5,height = 4,dpi = 300)

In [ ]:
options(repr.plot.width=3.5,repr.plot.height=4)
VlnPlot(test_g,group.by = "SCT_snn_res.0.4",
        features = c('Gfap'),ncol = 1,
        col = c('#618685',
         '#f2e394',
        '#f9d5e5',
        '#80ced6',
        '#eea29a'),
        #slot = "scale.data",
        pt.size = 0)+
    geom_boxplot(width=0.2,fill="white",outlier.colour = NA)
ggsave("Astrocyte.Gfap.subset.pc10.res04.Gfap.data.vlnplot.png",width = 3.5,height = 4,dpi = 300)

In [ ]:
options(repr.plot.width=3.5,repr.plot.height=4)
VlnPlot(test_g,group.by = "SCT_snn_res.0.4",
        features = c('Icam1'),ncol = 1,
        col = c('#618685',
         '#f2e394',
        '#f9d5e5',
        '#80ced6',
        '#eea29a'),
        #slot = "scale.data",
        pt.size = 0)+
    geom_boxplot(width=0.2,fill="white",outlier.colour = NA)
ggsave("Astrocyte.Gfap.subset.pc10.res04.Icam1.data.vlnplot.png",width = 3.5,height = 4,dpi = 300)

In [ ]:
options(repr.plot.width=3.5,repr.plot.height=4)
VlnPlot(test_g,group.by = "SCT_snn_res.0.4",
        features = c('Igfbp2'),ncol = 1,
        col = c('#618685',
         '#f2e394',
        '#f9d5e5',
        '#80ced6',
        '#eea29a'),
        #slot = "scale.data",
        pt.size = 0)+
    geom_boxplot(width=0.2,fill="white",outlier.colour = NA)
ggsave("Astrocyte.Gfap.subset.pc10.res04.Igfbp2.data.vlnplot.png",width = 3.5,height = 4,dpi = 300)

In [ ]:
options(repr.plot.width=8,repr.plot.height=8)
VlnPlot(test_g,group.by = "SCT_snn_res.0.3",cols = c('#f9d5e5',
        '#80ced6',
        '#eea29a'),
        features = c('Slc39a14','Bdkrb2','H2-K1','Lrrc8a','Slc5a3','Tubb2a','Lgals3bp','Ptprj','Csf1','Pttg1ip','Osmr','Icam1','Stt3b'),ncol = 4,
        #cols = c('#911eb4',"#ff7b25"),
        #slot = "scale.data",
        pt.size = 0)

In [308]:
Idents(test_g)<-test_g$SCT_snn_res.0.4

In [ ]:
mark<-FindAllMarkers(test_g,logfc.threshold = 0.25,min.pct = 0.5,
                     only.pos = T,return.thresh = 0.01)


In [363]:
write.csv(mark,"Astrocyte.Gfap.subset.pc10.res04.5clusters.fc05.pct05.marker.csv")

In [ ]:
options(repr.plot.width=6,repr.plot.height=6)
mark %>%
    group_by(cluster) %>%
    top_n(n = 10, wt = avg_log2FC) -> top10
DoHeatmap(test_g, features = top10$gene,size = 5,group.colors = c('#618685',
         '#f2e394',
        '#f9d5e5',
        '#80ced6',
        '#eea29a')) +
    scale_fill_gradientn(colours = c('#4575B4','#91BFDB','#E0F3F8','white','#FC8D59','#D73027'))#+ NoLegend()
ggsave("Astrocyte.Gfap.subset.pc10.res04.5clusters.fc05.pct05.pad001.marker.top10.heatmap.png",width = 6,height = 6,dpi = 300)

In [ ]:
backgenes<-read.table("/result/Seurat/sc_Astrocyte_recluster/lee/classifyBySpatialMarker/L2.Astrocyte.nC2500.rmMt_Rp.SCTregICMR.doscale.background.gene.ENTREZID.txt")
backgenes<-as.character(backgenes$x)
head(backgenes)

In [ ]:
library(org.Mm.eg.db)
library(ggplot2)
library(biomaRt)
#grlabs<-split(gene.nodle.df$gene,gene.nodle.df$module)
grlabs<-split(mark$gene,mark$cluster)
gcSample = lapply(grlabs, function(gr) as.numeric(bitr(gr, fromType = "SYMBOL", toType = "ENTREZID", OrgDb = "org.Mm.eg.db")$ENTREZID))
pvalueCutoff = 0.01
qvalueCutoff = 0.01
xx.mus.go2 <- compareCluster(gcSample, OrgDb='org.Mm.eg.db', fun='enrichGO', 
                            pvalueCutoff = pvalueCutoff, qvalueCutoff = qvalueCutoff, ont = "BP", 
                            readable=T,universe=backgenes,maxGSSize=150
                           )
#EnrichResGO_mm = xx.mus.go@compareClusterResult
#options(repr.plot.width=23,repr.plot.height=22)
#                  dotplot(xx.mus.go, title = paste0("Mouse Gene Ontology (qval < ", qvalueCutoff, ")"))
#saveRDS(xx.mus.go,"pc20.k50.res02.cluster_fc025.pct025.pos.markergene_pqv001.max100.go_220524.rds")


In [425]:
saveRDS(xx.mus.go2,"Astrocyte.Gfap.subset.pc10.res04.5clusters.fc05.pct05.pad001.marker.pqval001.max150.go.rds")

In [ ]:
options(repr.plot.width=6,repr.plot.height=10)
dotplot(xx.mus.go2,showCategory=10)

In [ ]:
df<-xx.mus.go2@compareClusterResult
head(df)
dim(df)
unique(df$Cluster)
table(df$Cluster)
library(dplyr)

In [432]:
df1 = df %>% group_by(.,Cluster) %>% filter(.,Count>=5) %>% top_n(.,-10,p.adjust) 
df1$Description = factor(df1$Description,levels = rev(unique(df1$Description)))              
                  

In [ ]:
options(repr.plot.width=8, repr.plot.height=6.5)
library(ggplot2)
p=ggplot(df1,aes(Cluster,Description))+geom_point(aes(size=Count,color=p.adjust))+    
    scale_color_gradientn(colors=brewer.pal(9,"Spectral"),guide = guide_colorbar(reverse = T))+
    theme_bw()+
    theme(axis.title = element_blank(),axis.text.y.left = element_text(size=13),
          axis.text.x.bottom = element_text(size = 13,angle = 45,hjust = 1,vjust = 1))
p
ggsave("Astrocyte.Gfap.subset.pc10.res04.5cluster.fc05.pct05.pad001.marker.pqval001.max150.go.C5.pad_top10.point.png",width = 8,height = 6.5,dpi = 300)


In [520]:
#saveRDS(test_g,"Astrocyte.Gfap.pc16.res02.rds")
saveRDS(test_g,"Astrocyte.Gfap.subset.pc10.res04.rds")

In [3]:
test_g<-readRDS("Astrocyte.Gfap.subset.pc10.res04.rds")

#### gfap clusters in astrocyte UMAP -220916

In [ ]:
temp<-test[rownames(test),colnames(test_g)]
temp

In [6]:
temp@meta.data<-test_g@meta.data

In [7]:
rd<-as.data.frame(temp@reductions$umap@cell.embeddings)

In [ ]:
cells<-rownames(rd)[rd$UMAP_1>-4 & rd$UMAP_2<4]
length(cells)

In [10]:
temp<-temp[rownames(temp),cells]

In [ ]:
pdf("Astrocyte.Gfap.subset.pc10.res04.time.UMAP_astro.220916.pdf",width = 3.6,height = 4.5)
options(repr.plot.width=3.6,repr.plot.height=4.5)
DimPlot(temp,reduction = "umap",pt.size = 1.3,#cols = DiscretePalette(n=10,palette = "glasbey"),#cells = cells,
        group.by = c("SCT_snn_res.0.4"))+
    scale_color_manual(values = c('#618685',
                                  '#f2e394',
        #'#d5f4e6',
        
        '#f9d5e5',
        '#80ced6',
        '#eea29a'

                                 ))
dev.off()
#ggsave("Astrocyte.Gfap.subset.pc10.res04.time.UMAP_astro.220916.png",width = 3.6,height = 4.5,dpi = 400)

### surface marker

In [ ]:
sf_gene<-read_excel("/result/Seurat/sc_Astrocyte_recluster/lee/S2_File.xlsx",sheet = "Table B")
head(sf_gene)

In [ ]:
# dp state surface markers
cand_sg<-intersect(mark[mark$cluster==3,"gene"],sf_gene$`ENTREZ gene symbol`)
cand_sg

In [ ]:
options(repr.plot.width=14,repr.plot.height=18)
VlnPlot(test_g,features = cand_sg,ncol = 8,
        cols = c(c('#4575B4','#91BFDB','#E0F3F8','white','#FC8D59','#D73027')),
        #slot = "scale.data",
        pt.size = 0)
ggsave("Astrocyte.Gfap.subset.pc10.res04.cluster3.fc05.pct05.pad001.surfacemarker.vlnplot.png",width = 14,height = 18,dpi = 300)